In [ ]:
%%sql
CREATE DATABASE geospatial;

## ColumnStore Limitations

In [ ]:
%%sql
CREATE TABLE neighborhoods_colstore(
  id INT UNSIGNED NOT NULL,
  name VARCHAR(64) NOT NULL,
  population INT UNSIGNED NOT NULL,
  shape TEXT NOT NULL, -- HERE YOU CANNOT USE GEOGRAPHY TYPE, BUT USE TEXT INSTEAD
  centroid GEOGRAPHYPOINT NOT NULL,
  sort key (name), shard key (id));

In [ ]:
%%sql
INSERT INTO neighborhoods_colstore VALUES
  (1, "Hamilton", 12346, "POLYGON((1 1,2 1,2 2, 1 2, 1 1))","POINT(1.5 1.5)"), -- NOTICE 4TH COLUMN IS A STRING
  (2, "Probability Bay", 263951, "POLYGON((5 1,6 1,6 2,5 2,5 1))", "POINT(5.5 1.5)"),
  (3, "Decentralized Park", 29265, "POLYGON((5 5,6 5,6 6,5 6,5 5))", "POINT(5.5 5.5)"),
  (4, "Axiom Township", 845696, "POLYGON((1 5,2 5,2 6,1 6,1 5))", "POINT(1.5 5.5)"),
  (5, "Elegant Island ", 987654, "POLYGON((3 3,4 3,4 4,3 4,3 3))", "POINT(3.5 3.5)");

## A RowStore table of all the neighborhoods

In [ ]:
%%sql
CREATE ROWSTORE TABLE neighborhoods (
  id INT UNSIGNED NOT NULL PRIMARY KEY,
  name VARCHAR(64) NOT NULL,
  population INT UNSIGNED NOT NULL,
  shape GEOGRAPHY NOT NULL,
  centroid GEOGRAPHYPOINT NOT NULL,
  index (shape) WITH (resolution = 8),
  index (centroid)
  );

In [ ]:
%%sql
INSERT INTO neighborhoods VALUES
  (1, "Hamilton", 12346, "POLYGON((1 1,2 1,2 2, 1 2, 1 1))","POINT(1.5 1.5)"),
  (2, "Probability Bay", 263951, "POLYGON((5 1,6 1,6 2,5 2,5 1))", "POINT(5.5 1.5)"),
  (3, "Decentralized Park", 29265, "POLYGON((5 5,6 5,6 6,5 6,5 5))", "POINT(5.5 5.5)"),
  (4, "Axiom Township", 845696, "POLYGON((1 5,2 5,2 6,1 6,1 5))", "POINT(1.5 5.5)"),
  (5, "Elegant Island ", 987654, "POLYGON((3 3,4 3,4 4,3 4,3 3))", "POINT(3.5 3.5)");

## Below finds a point and sees if the POINT intersects with any edge of the POLYGON

In [ ]:
%%sql
SELECT * FROM neighborhoods;

In [ ]:
%%sql
SELECT id, name FROM neighborhoods
WHERE GEOGRAPHY_INTERSECTS("POINT(3.94990499 3.69150746)", shape);

## A RowStore table of all businesses near the neighborhoods

In [ ]:
%%sql
CREATE ROWSTORE TABLE geo_businesses (
    id INT UNSIGNED NOT NULL PRIMARY KEY,
    name VARCHAR(64) NOT NULL,
    revenue INT UNSIGNED NOT NULL,
    location GEOGRAPHYPOINT NOT NULL,
    INDEX (location)
);

In [ ]:
%%sql
INSERT INTO geo_businesses VALUES
   (1,"Touring Machines Bike Repair", 10000, "POINT(1.25 1.25)"),
   (2,"Manhattan Boat Club", 20000, "POINT(5.75 5.25)"),
   (3,"The Philospher's Diner", 12345, "POINT(1.25 5.25)"),
   (4,"Push & Pop Cafe", 23456, "POINT(3.25 3.75)"),
   (5,"Lil Mac's Subs", 76543, "POINT(5.75 5.75)"),
   (6,"Touring Machines Bike Repair", 21012, "POINT(5.25 1.75)"),
   (7,"Random Slice Pizza", 11111, "POINT(1.75 1.75)")
;

## Below finds businesses contained within a specific neighborhood after joining businesses and neighborhoods table

In [ ]:
%%sql
SELECT b.id, b.name, b.location FROM geo_businesses b, neighborhoods n
WHERE n.name="Elegant Island" AND
GEOGRAPHY_CONTAINS(n.shape, b.location);

In [ ]:
%%sql
SELECT b.id,b.name, b.location, n.name, n.shape FROM geo_businesses b, neighborhoods n where b.id = 4 ORDER BY b.id desc;

## This is the Visual Representation of Elegant Island

![Elegant Island](https://github.com/bbohans2/singlestore-tutorials/raw/main/Photos/Elegant%20Island.png)

## This is the Visual Representation of Probability Bay

![Probability Bay](https://github.com/bbohans2/singlestore-tutorials/raw/main/Photos/Probability%20Bay.png)